In [35]:
import pandas as pd
import os
import geopandas as gpd
import glob
import plotly.express as px
import plotly.graph_objects as go
import folium
import json
import math
import numpy as np
import json
import requests

os.chdir("/home/ubuntu/realestate")
print(os.getcwd())

/home/ubuntu/realestate


In [36]:
api_key = '66be80af3672f1fd2d43413af6e1588b'

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    lat_lon = pd.DataFrame(match_first, index = [0])
    lat_lon = lat_lon[['x','y']]
    lat_lon['주소'] = addr
    return lat_lon


In [37]:
AptRent_Total = pd.read_csv("data/AptRent_Total.csv")
AptTrade_Total = pd.read_csv("data/AptTrade_Total.csv")
regal_code = pd.read_csv('data/regal_code.csv')



In [38]:
AptRent_Total = AptRent_Total.drop(['갱신요구권사용','계약구분','종전계약보증금','종전계약월세'], axis=1)
AptTrade_Total = AptTrade_Total.drop(['거래유형','중개사소재지','해제사유발생일','해제여부'], axis=1)

In [39]:
regal_code = regal_code.rename(columns={"법정동시군구코드":"지역코드"})
regal_code = regal_code.where(pd.notnull(regal_code), " ")

regal_code['시도명'] = regal_code['시도명'].str.strip()
regal_code['시군구명'] = regal_code['시군구명'].str.strip()
regal_code['읍면동명'] = regal_code['읍면동명'].str.strip()
regal_code['동리명'] = regal_code['동리명'].str.strip()

regal_code['주소'] = regal_code['시도명'] + " " + \
                       regal_code['시군구명'] + " " + \
                       regal_code['읍면동명'] + " " + \
                       regal_code['동리명']

regal_code['주소'] = regal_code['주소'].str.replace('  ', ' ')
regal_code['주소'] = regal_code['주소'].str.strip()
regal_code

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,지역코드,법정동읍면동코드,주소
0,1111010100,서울특별시,종로구,청운동,,19880423,,11110,10100,서울특별시 종로구 청운동
1,1111010200,서울특별시,종로구,신교동,,19880423,,11110,10200,서울특별시 종로구 신교동
2,1111010300,서울특별시,종로구,궁정동,,19880423,,11110,10300,서울특별시 종로구 궁정동
3,1111010400,서울특별시,종로구,효자동,,19880423,,11110,10400,서울특별시 종로구 효자동
4,1111010500,서울특별시,종로구,창성동,,19880423,,11110,10500,서울특별시 종로구 창성동
...,...,...,...,...,...,...,...,...,...,...
20273,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,,50130,32022,제주특별자치도 서귀포시 표선면 하천리
20274,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,,50130,32023,제주특별자치도 서귀포시 표선면 성읍리
20275,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,,50130,32024,제주특별자치도 서귀포시 표선면 가시리
20276,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,,50130,32025,제주특별자치도 서귀포시 표선면 세화리


### 법정동 코드와 merge

- 지역코드 를 key로 merge하여 시군구명과 주소를 붙힘

#### 1차 머지 주소만 생성

In [40]:
regal_key_value = regal_code[['지역코드', '시군구명']].drop_duplicates().reset_index(drop = True)

In [41]:
AptRent_Total_2 = pd.merge(AptRent_Total,regal_key_value,
                            on = '지역코드',
                            how = 'left')

AptRent_Total_2['주소'] = "서울특별시 " + AptRent_Total_2['시군구명'] + " " + \
                       AptRent_Total_2['법정동']  + " " + \
                       AptRent_Total_2['지번']  + " " + \
                       AptRent_Total_2['아파트']


In [42]:
AptTrade_Total_2 = pd.merge(AptTrade_Total,regal_key_value,
                            on = '지역코드',
                            how = 'left')

AptTrade_Total_2['주소'] = "서울특별시 " + AptTrade_Total_2['시군구명'] + " " + \
                       AptTrade_Total_2['법정동']  + " " + \
                       AptTrade_Total_2['지번']  + " " + \
                       AptTrade_Total_2['아파트']


In [43]:
len(AptTrade_Total_2['주소'].unique())

7663

### 2차 머지: 주소를 고유값으로 변경 하여, 위경도 생성

In [44]:
AptTrade_addr_list = AptTrade_Total_2['주소'].unique()

In [45]:
AptRent_addr_list = AptRent_Total_2['주소'].unique()

In [46]:
total_addr = pd.DataFrame()
# 주소 고유값 사용
for addr in AptTrade_addr_list:
    print(addr)
    # 에러 발생 방지 위해 try 함수 사용
    try:
        # addr_to_lat_lon함수를 사용하여 위경도 추출
        df = addr_to_lat_lon(addr_to_lat_lon)
        # pd.concat를 사용하여 total_addr에 붙힘
        total_addr = pd.concat([total_addr, df])
    except:
        df = pd.DataFrame({'x': '0',
                           'y': '0',
                          '주소': addr})
        total_addr = pd.concat([total_addr, df])

        

서울특별시 종로구  필운동 254 신동아블루아광화문의꿈


ValueError: If using all scalar values, you must pass an index

In [13]:
# 새로운 데이터 생성
AptTrade_addr_df = total_addr
AptTrade_addr_df.to_csv('AptTrade_addr_df.csv', index = False)

NameError: name 'total_addr' is not defined

## merge 2


In [33]:
AptTrade_Total_3 = pd.merge(AptTrade_Total_2, AptTrade_addr_df,
        on = '주소'
        how = 'left')

SyntaxError: invalid syntax (1437177212.py, line 3)

In [ ]:
AptTrade_Total_3.to_csv('AptTrade_Total_3.csv', index = False)

In [ ]:
total_addr = pd.DataFrame()
# 주소 고유값 사용
for addr in AptRent_addr_list:
    print(addr)
    # 에러 발생 방지 위해 try 함수 사용
    try:
        # addr_to_lat_lon함수를 사용하여 위경도 추출
        df = addr_to_lat_lon(addr_to_lat_lon)
        # pd.concat를 사용하여 total_addr에 붙힘
        total_addr = pd.concat([total_addr, df])
    except:
        df = pd.DataFrame({'x': '0',
                           'y': '0',
                          '주소': addr})
        total_addr = pd.concat([total_addr, df])

# 새로운 데이터 생성
AptRent_addr_df = total_addr
AptRent_addr_df.to_csv('AptRent_addr_df.csv', index = False)        

In [ ]:

AptRent_Total_3 = pd.merge(AptRent_Total_2, AptRent_addr_df,
        on = '주소'
        how = 'left')

In [ ]:
AptRent_Total_3.to_csv('AptRent_Total_3.csv', index = False)